In [1]:
import pandas as pd

In [3]:
# read first csv file
df1 = pd.read_csv('../GVC_data/eora_052423.csv')

# read second csv file
df2 = pd.read_csv('../stability/stability_df.csv')

# merge the two dataframes based on year and country columns
merged_df = pd.merge(df1, df2, on=['t', 'country'])

sorted_df = merged_df.sort_values(['country', 'sect', 't'], ascending=[True, True, True])

# save the merged dataframe to a new csv file
sorted_df.to_csv('merged_052423.csv', index=False)

In [5]:
def process_dataframe(df, value_column):
    # Pivot the dataframe to have one row per country and year with columns for each sector
    df_pivot = df.pivot_table(index=['country', 't'], columns='sect', values=value_column+'_diff', aggfunc='first').reset_index()

    # Flatten the MultiIndex in columns
    df_pivot.columns = [col if not isinstance(col, tuple) else f"{col[0]} {col[1]}" for col in df_pivot.columns]

    # Merge the pivoted dataframe with the unique dataframe
    df_final = pd.merge(df_pivot, df_unique, on=['country', 't'], how='left')

    # Rename the columns
    df_final = df_final.rename(columns={1: 's1',
                                        2: 's2',
                                        6: 's6',
                                        10: 's10',
                                        19: 's19'})

    # Calculate the averages for each column by year
    avg_df = df_final.groupby('t')[['s1', 's2', 's6', 's10', 's19']].mean()

    # Rename the columns to include 'avg' in the name
    avg_df = avg_df.add_prefix('avg')

    # Merge the original data frame with the new average data frame on the year column
    result = pd.merge(df_final, avg_df, on='t')

    return result

# Read the CSV file
df = pd.read_csv('merged_052423.csv')

# Calculate year-on-year differences for democracy
df['democracy_diff'] = df.groupby('country')['democracy'].diff()

trans_IV = pd.read_csv('../GVC_data/transportIV_file.csv')
trans_IV.drop(['outp', 'gross output'], axis=1, inplace=True)

# List of additional columns to include in the final output
additional_columns = ['coup', 'onset2COWCS', 'decade', 'ecgrowth', 'logpop_M', 'logpopdens', 'logoutreg', 'democracy', 'logmountain', 
                      'ethnic_fractionalization', 'religion_fractionalization', 'language_fractionalization', 'leg_british', 'opec',
                      'logpop_M_diff', 'logpopdens_diff', 'logoutreg_diff', 'ecgrowth_demeaned', 'democracy_diff']

# Drop duplicates in the original dataframe based on 'country', 'year', and additional_columns
df_unique = df[['country', 't'] + additional_columns].drop_duplicates()

result_gvcomix = process_dataframe(df, 'gvcomix')
result_gvcobp = process_dataframe(df, 'gvcobp')
result_gvcofp = process_dataframe(df, 'gvcofp')

# Now merge with the transport IV dataframe
result_gvcomix = pd.merge(result_gvcomix, trans_IV, on=['country', 't'])
result_gvcobp = pd.merge(result_gvcobp, trans_IV, on=['country', 't'])
result_gvcofp = pd.merge(result_gvcofp, trans_IV, on=['country', 't'])

# Save the transformed dataframes to new CSV files
result_gvcomix.to_csv('gvcomix_data.csv', index=False)
result_gvcobp.to_csv('gvcobp_data.csv', index=False)
result_gvcofp.to_csv('gvcofp_data.csv', index=False)